***Retrieval QA and self query filtering***

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

***Unstructured Excel Loader***

Unstructured store in elements format, and langchin converts to document format
Retrieve html format from the table metadata to get the table information

In [19]:
extracted_elements = []
from langchain_community.document_loaders import UnstructuredExcelLoader
loader = UnstructuredExcelLoader("xlfiles/mock_recomm_library_latest.xlsx", mode="elements")
documents = loader.load()
for doc in documents:
    extracted_elements.append(doc.metadata['text_as_html'])

# Write the extracted elements to the output file
with open("mock_recomm_lib.txt", 'w') as output_file:
    for element in extracted_elements:
        output_file.write(element + "\n\n") # Adding two newlines for separation

TextLoader to load text files

In [20]:
from langchain.document_loaders import TextLoader
loader = TextLoader("mock_recomm_lib.txt")
text_documents = loader.load()

Chunking - RecursiveCharacter Splitter 

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split it into chunks
# text_splitter = RecursiveCharacterTextSplitter(
# chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
# )
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(text_documents)
for doc in docs:
    print(doc.page_content)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Product/ Reporting Category*</td>
      <td>Observation</td>
      <td>Recommendation 1</td>
      <td>Recommendation 2</td>
      <td>Recommendation 3</td>
      <td>Why</td>
      <td>Themes</td>
    </tr>
    <tr>
      <td>HealthHive Preventive Care Kit</td>
      <td>70% of our 10,000 members have reported difficulty keeping track of recommended health screenings and vaccinations.</td>
      <td>Use HealthHive to schedule and track preventive care appointments.\nReceive personalized reminders for recommended screenings and vaccinations.\nAccess educational resources to understand the importance of preventive care.</td>
      <td></td>
      <td></td>
      <td>Regular screenings detect health issues early.\nVaccinations prevent illnesses and outbreaks.\nEducation empowers informed health decisions.</td>
      <td>Preventative Care</td>
    </tr>
    <tr>
      <td>CareConnect Access Platform</td>
<td>Preventative Ca

Create Embeddings to store in Vector DB

In [113]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Use Chroma as Vector storage

In [24]:
import chromadb
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

Prepare Prompt Template to instruct LLM

In [99]:
Prompt = """
            Please use below information and find recommendations only for the user asked Theme & product
            1) HealthHive Preventive care Kit Product/ Reporting Category and HealthHero Preventive Care Chatbot  Product/ Reporting Category are products for Preventive care theme.
            2) CareBridge Access to Care Platform  Product/ Reporting Category and Careconnect Access Platform Kit Product/ Reporting Category are products for Access to care theme.
            3) MindWave Behavioral Kit Product/ Reporting Category is a product for Behavioral theme.
            4) ChronicCare Program Kit Product/ Reporting Category is a product for Chronic theme
            5) Specialityscript Medical Kit Product/ Reporting Category is a product for Pharmacy theme.
        
        Please review the question if it matches with any of the themes listed and provide answer only for the user asked Theme & product in json including Theme, Product and Recommedation as dictionary keys 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. 

        \n----------------\n{context}
       
        """

Use Retrieval QA

In [114]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# Initialize your model and retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever(), verbose=True)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever(), verbose=True, chain_type_kwargs={"prompt": PromptTemplate(input_variables=['context'], template=Prompt)})


In [119]:
# List of questions
questions = [
"What are the recommendations for preventive care?"
# "What are the recommendations for Access to care?",
# "What are the products for Access to care?"
]

In [120]:
# Store responses in output_list
output_list = []

for query in questions:
    response = qa_chain({"query": query})
    output_list.append(response)



> Entering new RetrievalQA chain...

> Finished chain.


In [117]:
for res in output_list:
    print(res['query'] + "\n" + res['result'])

What are the recommendations for preventive care?
{
    "Theme": "Preventative Care",
    "Product": "HealthHive Preventive Care Kit",
    "Recommendation": {
        "1": "Use HealthHive to schedule and track preventive care appointments.",
        "2": "Receive personalized reminders for recommended screenings and vaccinations.",
        "3": "Access educational resources to understand the importance of preventive care."
    }
}


GPT 4 

In [118]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# Initialize your model and retriever
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
# qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever(), verbose=True)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever(), verbose=True, chain_type_kwargs={"prompt": PromptTemplate(input_variables=['context'], template=Prompt)})


In [121]:
for res in output_list:
    print(res['query'] + "\n" + res['result'])

What are the recommendations for preventive care?
{
"Theme": "Preventative Care",
"Product": [
{
"Name": "HealthHive Preventive Care Kit",
"Recommendation": "Use HealthHive to schedule and track preventive care appointments.\nReceive personalized reminders for recommended screenings and vaccinations.\nAccess educational resources to understand the importance of preventive care."
},
{
"Name": "HealthHero Preventive Care Chatbot",
"Recommendation": "Use HealthHero to ask questions and receive personalized preventive care recommendations.\nAccess educational resources to understand the importance of preventive care.\nSchedule appointments with healthcare providers through the chatbot."
}
]
}
